In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def print_usage():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
print_usage()

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My\ Drive/AI\ Project

Don't run this mkdir block now

In [ ]:
# mkdir saved_dqn_cnn-updated

In [ ]:
# !ls

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

**New Environment**

In [ ]:
#@title
from copy import copy


class ArrayError(Exception):
    pass


class Array(object):
    def __init__(self, width, height, empty=None):
        self._width = width
        self._height = height
        self._empty = empty

        self._reset()

    def _reset(self, value=None):
        value = value or self._empty

        self._array = [
            [value for i in range(self._width)]
            for j in range(self._height)
        ]

    def _check_index(self, x, y):
        if (
            x < 0 or
            x >= self._width or
            y < 0 or
            y >= self._height
        ):
            raise ArrayError('Index is not within array dimensions {w}x{h}'.format(
                x=x, y=y, w=self._width, h=self._height
            ))

    def _zero_index(cls, x, y):
      return x, y

    def __getitem__(self, i):
        self._check_index(*i)
        x, y = self._zero_index(*i)
        return self._array[y][x]

    def __setitem__(self, i, value):
        self._check_index(*i)
        x, y = self._zero_index(*i)
        self._array[y][x] = value

    def __eq__(self, other):
        return self._array == other._array

    @property
    def copy(self):
        new = copy(self)
        new._array = [copy(row) for row in self._array]
        return new

In [ ]:
#@title
from collections import namedtuple
from copy import copy

# from .array import Array, ArrayError
# from .location import Location


class BoardError(Exception):
    pass


class Board(Array):
    """
    Stores board locations.  Provides methods to carry out game logic.
    """
    BLACK = 1
    WHITE = 2
    EMPTY = 0

    TURNS = (
        BLACK,
        WHITE,
    )
    KOMI = 0

    State = namedtuple('State', ['board', 'turn', 'score'])

    def __init__(self, width,komi=0):
        super(Board, self).__init__(width, width, self.EMPTY)

        # Turn counter
        self._turn = self.BLACK
        self.width = width
        self.KOMI = komi

        # Player scores
        self._score = {
            self.BLACK: 0,
            self.WHITE: 0,
        }

        # Game history
        self._history = []
        self._redo = []

    @property
    def turn(self):
        """
        Gets the current turn.
        """
        return repr(self._turn)

    @property
    def score(self):
        """
        Gets the current score.
        """
        return {
            'black': self._score[self.BLACK],
            'white': self._score[self.WHITE],
        }
      
    def is_eyeish(self, x, y, owner):
      if self[x, y] != self.EMPTY:
        return False
      
      for p, (x1, y1) in self._get_surrounding(x, y):
        if self[x1,y1] != owner:
          return False
      return True
    
    def territory_score(self):
      
      score = {
          self.BLACK : 0,
          self.WHITE : 0,
      }
      
      traversed = set()
      
      for i in range(self.width):
        for j in range(self.width):
          
          if(self[i,j]== self.EMPTY and (i,j) not in traversed):
            nei = set()
            group = self._get_group(i, j, set())
            
            for a,b in group:
              
              sur = self._get_surrounding(a, b)
              
              for p, (k, l) in sur:
                if(p != self.EMPTY):
                  nei.add(p)
#             print(nei)
            
            if(len(nei)==1):
              score[list(nei)[0]] += len(group)
            traversed = traversed.union(group)
      return score
              
              
            
     
    def board_score(self):
        """
        Gets the current score.
        """
        initial_score = copy(self._score)
        
#         initial_score = {
#             self.BLACK: 0,
#             self.WHITE: 0,
#         }
        
#         for i in range(self.width):
#           for j in range(self.width):
#             if(self[i,j]==self.EMPTY):
#               if(self.is_eyeish(i, j, self.BLACK)):
#                 initial_score[self.BLACK] += 1
#               elif(self.is_eyeish(i, j, self.WHITE)):
#                 initial_score[self.WHITE] += 1
#             else:
# #               print('score added for',self[i,j])
#               initial_score[self[i,j]] += 1
              
#         initial_score[self.WHITE] += 6.5
        for i in range(self.width):
          for j in range(self.width):
            if(self[i,j]!=self.EMPTY):
              initial_score[self[i,j]] +=1
        
        terr = self.territory_score()
        
        initial_score[self.BLACK] += terr[self.BLACK]
        initial_score[self.WHITE] += terr[self.WHITE]
        
        initial_score[self.WHITE] += self.KOMI
        
        return initial_score

    @property
    def _next_turn(self):
        """
        Gets color of next turn.
        """
        return self.TURNS[self._turn is self.BLACK]

    def move(self, x, y, passMove = False):
        """
        Makes a move at the given location for the current turn's color.
        """
        if passMove:
          self._flip_turn()
          return 0
        # Check if coordinates are occupied
        if self[x, y] is not self.EMPTY:
#           raise BoardError('Cannot move on top of another piece!')
#           print("x=",x,' y=',y)
#           self.printBoard()
#           print("not empty block")
          return 1

        # Store history and make move
        self._push_history()
        self[x, y] = self._turn

        # Check if any pieces have been taken
        taken = self._take_pieces(x, y)

        # Check if move is suicidal.  A suicidal move is a move that takes no
        # pieces and is played on a coordinate which has no liberties.
        if taken == 0:
#             self.printBoard()
            suicide = self._check_for_suicide(x, y)
            if(suicide==1):
#               print("tried suicide")
              return 1

        # Check if move is redundant.  A redundant move is one that would
        # return the board to the state at the time of a player's last move.
        koMove = self._check_for_ko()
        if(koMove == 1):
#           print("tried ko")
          return 1

        self._flip_turn()
        self._redo = []
        return 0
      
    def printBoard(self):
      tt = ''
      for i in range(self.width):
        for j in range(self.width):
          tt += str(self[i,j]) + ' '
        tt += '\n'
      print(tt)
      
    def find_legal_moves(self):
      moves = []
      for i in range(self.width):
        for j in range(self.width):
          if(self[i,j] is not self.EMPTY):
            continue
          else:
            group=[]
            self[i,j] = self._turn
            taken,group = self.check_take_pieces(i,j)
#             print('taken=',taken)
            
            if(taken==0):
              suicide = self._check_for_suicide_nochange(i,j)
              if(suicide==1):
                self[i,j] = self.EMPTY
                continue
            
            for x1, y1 in group:
              self[x1, y1] = self.EMPTY

            koMove = self._check_for_ko_nochange()
            
            if(koMove == 1):
              self[i,j] = self.EMPTY
              for x1,y1 in group:
                  self[x1,y1] = self._next_turn
              continue
#             print("next turn =",self._next_turn)
#             print("group",group)
            for x1,y1 in group:
                  self[x1,y1] = self._next_turn
            
            self[i,j] = self.EMPTY
            moves.append(i*self.width+j)
      return moves

    def _check_for_suicide_nochange(self, x, y):
      """
      Checks if move is suicidal.
      """
      if self.count_liberties(x, y) == 0:
#           self._pop_history()
#             raise BoardError('Cannot play on location with no liberties!')
          return 1

      return 0

    
    def _check_for_suicide(self, x, y):
        """
        Checks if move is suicidal.
        """
        if self.count_liberties(x, y) == 0:
            self._pop_history()
#             raise BoardError('Cannot play on location with no liberties!')
            return 1

        return 0

    def _check_for_ko_nochange(self):
        """
        Checks if board state is redundant.
        """
        try:
            if self._array == self._history[-1][0]:
#                 self._pop_history()
#                 raise BoardError('Cannot make a move that is redundant!')
                return 1
        except IndexError:
            # Insufficient history...let this one slide
            pass

    def _check_for_ko(self):
        """
        Checks if board state is redundant.
        """
        try:
            if self._array == self._history[-2][0]:
                self._pop_history()
#                 raise BoardError('Cannot make a move that is redundant!')
                return 1
        except IndexError:
            # Insufficient history...let this one slide
            pass

    def _take_pieces(self, x, y):
        """
        Checks if any pieces were taken by the last move at the specified
        coordinates.  If so, removes them from play and tallies resulting
        points.
        """
        scores = []
        for p, (x1, y1) in self._get_surrounding(x, y):
            # If location is opponent's color and has no liberties, tally it up
            if p is self._next_turn and self.count_liberties(x1, y1) == 0:
                score = self._kill_group(x1, y1)
                scores.append(score)
                self._tally(score)
        return sum(scores)
      
    def check_take_pieces(self,x,y):
#         scores = []
        gg = []
        sco = 0
        for p, (x1, y1) in self._get_surrounding(x, y):
            # If location is opponent's color and has no liberties, tally it up
            if p is self._next_turn and self.count_liberties(x1, y1) == 0:
                score,group = self.check_kill_group(x1, y1)
                if(score>0):
                  gg += group
                  sco+=score
        
        if(sco>0):
          return sco,gg
#         scores.append(score)
#         self._tally(score)
        return 0,gg

    def _flip_turn(self):
        """
        Iterates the turn counter.
        """
        self._turn = self._next_turn
        return self._turn

    @property
    def _state(self):
        """
        Returns the game state as a named tuple.
        """
        return self.State(self.copy._array, self._turn, copy(self._score))

    def _load_state(self, state):
        """
        Loads the specified game state.
        """
        self._array, self._turn, self._score = state

    def _push_history(self):
        """
        Pushes game state onto history.
        """
        self._history.append(self._state)

    def _pop_history(self):
        """
        Pops and loads game state from history.
        """
        current_state = self._state
#         print('was called')
        try:
            self._load_state(self._history.pop())
            return current_state
        except IndexError:
            return None

    def undo(self):
        """
        Undoes one move.
        """
        state = self._pop_history()
        if state:
            self._redo.append(state)
            return state
        else:
            raise BoardError('No moves to undo!')

    def redo(self):
        """
        Re-applies one move that was undone.
        """
        try:
            self._push_history()
            self._load_state(self._redo.pop())
        except IndexError:
            self._pop_history()
            raise BoardError('No undone moves to redo!')

    def _tally(self, score):
        """
        Adds points to the current turn's score.
        """
        self._score[self._turn] += score

    def _get_none(self, x, y):
        """
        Same thing as Array.__getitem__, but returns None if coordinates are
        not within array dimensions.
        """
        try:
            return self[x, y]
        except ArrayError:
#             print("ARRAY ERROR")
            return None

    def _get_surrounding(self, x, y):
        """
        Gets information about the surrounding locations for a specified
        coordinate.  Returns a tuple of the locations clockwise starting from
        the top.
        """
        coords = (
            (x, y - 1),
            (x + 1, y),
            (x, y + 1),
            (x - 1, y),
        )
#         return filter(lambda i: bool(i[0]), [
#             (self._get_none(a, b), (a, b))
#             for a, b in coords
#         ])
        tt = []
        for a,b in coords:
        
          if(a>=0 and a<self.width and b>=0 and b<self.width):
#             print('a=',a,'b=',b)
            tt.append((self[a,b],(a,b)))
                      
#           else:
#             tt.append((None,(a,b)))
#         print('tt=',tt)
#         print('ff=',filter(lambda i: bool(i[0]),tt))
#         return filter(lambda i: bool(i[0]),tt)
        return tt

    def _get_group(self, x, y, traversed):
        """
        Recursively traverses adjacent locations of the same color to find all
        locations which are members of the same group.
        """
        loc = self[x, y]

        # Get surrounding locations which have the same color and whose
        # coordinates have not already been traversed
        locations = [
            (p, (a, b))
            for p, (a, b) in self._get_surrounding(x, y)
            if p is loc and (a, b) not in traversed
        ]

        # Add current coordinates to traversed coordinates
        traversed.add((x, y))

        # Find coordinates of similar neighbors
        if locations:
            return traversed.union(*[
                self._get_group(a, b, traversed)
                for _, (a, b) in locations
            ])
        else:
            return traversed

    def get_group(self, x, y):
        """
        Gets the coordinates for all locations which are members of the same
        group as the location at the given coordinates.
        """
        if self[x, y] not in self.TURNS:
            raise BoardError('Can only get group for black or white location')

        return self._get_group(x, y, set())

    def _kill_group(self, x, y):
        """
        Kills a group of black or white pieces and returns its size for
        scoring.
        """
        if self[x, y] not in self.TURNS:
            raise BoardError('Can only kill black or white group')

        group = self.get_group(x, y)
        score = len(group)

        for x1, y1 in group:
            self[x1, y1] = self.EMPTY

        return score
      
    def check_kill_group(self, x, y):
        if self[x, y] not in self.TURNS:
            return 0

        group = self.get_group(x, y)
        score = len(group)
        
#         for x1, y1 in group:
#             self[x1, y1] = self.EMPTY

        return score,group

    def _get_liberties(self, x, y, traversed):
        """
        Recursively traverses adjacent locations of the same color to find all
        surrounding liberties for the group at the given coordinates.
        """
        loc = self[x, y]
#         self.printBoard()
#         print('x=',x,'y=',y)
#         print('loc=',loc)
#         print('sur=',self._get_surrounding(x,y))

        if loc is self.EMPTY:
            # Return coords of empty location (this counts as a liberty)
            return set([(x, y)])
        else:
            # Get surrounding locations which are empty or have the same color
            # and whose coordinates have not already been traversed
            locations = [
                (p, (a, b))
                for p, (a, b) in self._get_surrounding(x, y)
                if (p is loc or p is self.EMPTY) and (a, b) not in traversed
            ]

            # Mark current coordinates as having been traversed
            traversed.add((x, y))

            # Collect unique coordinates of surrounding liberties
            if locations:
                return set.union(*[
                    self._get_liberties(a, b, traversed)
                    for _, (a, b) in locations
                ])
            else:
                return set()

    def get_liberties(self, x, y):
        """
        Gets the coordinates for liberties surrounding the group at the given
        coordinates.
        """
        return self._get_liberties(x, y, set())

    def count_liberties(self, x, y):
        """
        Gets the number of liberties surrounding the group at the given
        coordinates.
        """
        return len(self.get_liberties(x, y))
      
    def has_ended(self):
        
        for i in range(self.width):
          for j in range(self.width):
            
            if(self[i,j]==self.EMPTY):
              
              t_black = 0
              t_white = 0

              self[i,j] = self.BLACK
              
              taken,group = self.check_take_pieces(i,j)
              
#               print('taken= ',taken)
              
              if(taken==0):
                  suicide = self._check_for_suicide_nochange(i,j)
#                   print("suic=",suicide)
                  if(suicide!=1):
#                     self[i,j] = self.EMPTY
                    t_black +=1
              for x1, y1 in group:
                self[x1, y1] = self.EMPTY
                  
              koMove = self._check_for_ko_nochange()
            
#               print("ko=",koMove)
              
              if(koMove != 1 or koMove == None):
                self[i,j] = self.EMPTY
                for x1, y1 in group:
                  self[x1, y1] = self.WHITE
                t_black +=1
#               print("t_black=",t_black)
                  
              if(t_black==2):
                self[i, j] = self.EMPTY
                return False
              
              for x1, y1 in group:
                self[x1, y1] = self.WHITE
              
              
              self[i,j] = self.WHITE
              taken,group = self.check_take_pieces(i,j)
              if(taken==0):
                suicide = self._check_for_suicide_nochange(i,j)
                if(suicide!=1):
#                   self[i,j] = self.EMPTY
                  t_white +=1
             
              for x1, y1 in group:
                self[x1, y1] = self.EMPTY
                  
              koMove = self._check_for_ko_nochange()
              if(koMove != 1):
                self[i,j] = self.EMPTY
                for x1, y1 in group:
                  self[x1, y1] = self.BLACK
                t_white +=1
              
              for x1, y1 in group:
                self[x1, y1] = self.BLACK
              
              if(t_white==2):
                self[i, j] = self.EMPTY
                return False
              
              self[i,j] = self.EMPTY
        return True
      
    def has_ended_any(self):
         
      mov_black = 0
      mov_white = 0

      for i in range(self.width):
        for j in range(self.width):

          if(self[i,j]==self.EMPTY):

            t_black = 0
            t_white = 0


            self[i,j] = self.BLACK

            taken,group = self.check_take_pieces(i,j)

            if(taken==0):
                suicide = self._check_for_suicide_nochange(i,j)
                if(suicide!=1):
#                     self[i,j] = self.EMPTY
                  t_black +=1
            for x1, y1 in group:
              self[x1, y1] = self.EMPTY

            koMove = self._check_for_ko_nochange()
            if(koMove != 1):
#                 self[i,j] = self.EMPTY
              t_black +=1

            if(t_black==2):
              mov_black +=1
            
            for x1, y1 in group:
              self[x1, y1] = self.WHITE


            self[i,j] = self.WHITE
            taken,group = self.check_take_pieces(i,j)
            if(taken==0):
              suicide = self._check_for_suicide_nochange(i,j)
              if(suicide!=1):
#                   self[i,j] = self.EMPTY
                t_white +=1
            
            for x1, y1 in group:
              self[x1, y1] = self.EMPTY
        
            koMove = self._check_for_ko_nochange()
            if(koMove != 1):
#                 self[i,j] = self.EMPTY
              t_white +=1

            if(t_white==2):
              mov_white +=1
            for x1, y1 in group:
              self[x1, y1] = self.BLACK
            
            self[i,j] = self.EMPTY

      if(mov_white>0 and mov_black>0):
        return False
      else:
        return True

Model

In [ ]:
import torchvision
import torch.nn as nn

BOARD_SIZE = 5
NUM_ACTIONS = BOARD_SIZE*BOARD_SIZE + 1

class DQN(nn.Module):
    def __init__(self, num_classes=5*5+1):
        super(DQN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(16),
            nn.ReLU())
#             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding = 1))
        self.fc1 = nn.Linear(((BOARD_SIZE+2)/2)*((BOARD_SIZE+2)/2)*32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = DQN(NUM_ACTIONS)
model.cuda()

In [ ]:
model

In [ ]:
import numpy as np
learning_rate = 1e-3
discount_factor = 0.2
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adadelta(model.parameters())

In [ ]:
def createBoardStates(num):
  board_states = []
  env_states = []
  for i in range(num):
    state = Board(BOARD_SIZE)
    env_states.append(copy(state))
#     print state
    state = np.array(state._array).T
#     print state.shape
    board_states.append(state)
    
  board_states = np.array(board_states)
  return board_states, env_states

In [ ]:
def createConvBoardStates(num):
  states, env_states = createBoardStates(num)
  outStates = []
  for state in states:
    oState = np.zeros((3,state.shape[0],state.shape[1]))
    for i in range(state.shape[0]):
      for j in range(state.shape[1]):
        oState[state[i,j],i,j] = 1
    outStates.append(oState)
  outStates = np.array(outStates)
  return outStates, env_states

In [ ]:
def convertEnvToStates(env_states):
  outStates = []
  for eState in env_states:
    state = np.array(eState._array).T
    oState = np.zeros((3,state.shape[0],state.shape[1]))
    for i in range(state.shape[0]):
      for j in range(state.shape[1]):
        oState[state[i,j],i,j] = 1
    outStates.append(oState)
  outStates = np.array(outStates)
  return outStates

In [ ]:
def convertEnvToOpponentStates(env_states):
  outStates = convertEnvToStates(env_states)
  outStates[:,[0,1,2]] = outStates[:,[0,2,1]]
  return outStates

In [ ]:
!free
print_usage()

In [ ]:
from copy import deepcopy
import torch
from torch.autograd import Variable

model.train()

BATCH_SIZE = 16
NUM_EPOCHS = 1000000
MAX_RANDOM = 1.
MIN_RANDOM = 0.01
DECAY_RATE = 0.001
DECAY_STEP = 0
# opponentModel = DQN(BOARD_SIZE)
# OPPONENT_POOL = [opponentModel.load_state_dict(deepcopy(model.state_dict()))]
opponentPool = []
saved_states = None
saved_env_states = None
for epoch in range(NUM_EPOCHS):
#   print epoch
  if((epoch % 50)==0):
    oppModel = DQN(NUM_ACTIONS).cuda()
#     print oppModel
#     print model
    oppModel.load_state_dict(deepcopy(model.state_dict()))
    oppModel.eval()
    opponentPool.append(oppModel)
#     oppModel

    if(len(opponentPool)>50):
      opponentPool[-50:]
  
  if(saved_states is None):
    board_states, env_states = createConvBoardStates(BATCH_SIZE)
  elif(saved_states.shape[0]<BATCH_SIZE):
    bStates, eStates = createConvBoardStates(BATCH_SIZE-saved_states.shape[0])
#     print saved_states.shape, bStates.shape
#     print len(saved_env_states)
#     print len(eStates)
    board_states, env_states = np.concatenate((saved_states, bStates)), np.concatenate((saved_env_states, eStates))
  else:
    board_states, env_states = saved_states, saved_env_states
    
  inputs = Variable(torch.tensor(board_states, dtype=torch.float).cuda())
  outputs = model(inputs)
  
  randomMove = np.random.rand(BATCH_SIZE)
  
#   outputsNumpy = outputs.data.cpu().numpy()
  labels = []
  rewards = []
  invalidIndices = []
  for i in range(BATCH_SIZE):
    RANDOM = MIN_RANDOM + (MAX_RANDOM - MIN_RANDOM)*np.exp(-DECAY_RATE*DECAY_STEP)
    if(randomMove[i]<RANDOM):
      #Make random move - explore
      legal_moves = env_states[i].find_legal_moves()
      if(len(legal_moves)==0):
        move = NUM_ACTIONS-1
      else:
        move = legal_moves[np.random.randint(len(legal_moves))]
    else:
      #Make optimal move - exploit
      _, move = outputs[i].max(0)
      move = move.data.cpu().numpy()
    labels.append(outputs[i,move])
#     print move
#     move = move.data.cpu().numpy()[0]
    invalid = env_states[i].move(int(move/BOARD_SIZE), move%BOARD_SIZE, passMove = (move == NUM_ACTIONS-1))
    if(invalid==1):
#       legal_moves = env_states[i].find_legal_moves()
#       if(len(legal_moves)==0):
#         move = NUM_ACTIONS-1
#       else:
#         move = legal_moves[np.random.randint(len(legal_moves))]
#       env_states[i].move(int(move/BOARD_SIZE), move%BOARD_SIZE, passMove = (move == NUM_ACTIONS-1))
      invalidIndices.append(i)
      rewards.append(-10. - outputs[i,move])
    else:
      score = env_states[i].board_score()
      rewards.append(torch.sign(torch.tensor(score[env_states[i].BLACK]-score[env_states[i].WHITE])))
#   labelsNumpy = np.array(labels)
  labelsStacked = torch.stack(labels)
  labels = Variable(labelsStacked.cuda(), requires_grad=True)
  
#   rewards.requires_grad = False
  saved_states = convertEnvToOpponentStates(env_states)
  DECAY_STEP += 1
  
  #Choose opponent model
#   print len(opponentPool)
  rand = np.random.randint(len(opponentPool))
#   print rand
#   with torch.no_grad():
  opponentModel = opponentPool[rand]
  new_inputs = Variable(torch.tensor(saved_states, dtype=torch.float).cuda())
  new_outputs = opponentModel(new_inputs)
    
#   new_outputs_numpy = new_outputs.data.cpu().numpy()
#   new_labels = []
  hadEnded = []
  hasEnded = []
  for i in range(BATCH_SIZE):
    if(i in invalidIndices):
      hadEnded.append(False)
      hasEnded.append(False)
      continue
    hadEnded.append(env_states[i].has_ended())
    if(hadEnded[i]):
      move = NUM_ACTIONS-1
      env_states[i].move(int(move/BOARD_SIZE), move%BOARD_SIZE, passMove = (move == NUM_ACTIONS-1))
    else:
      _, move = new_outputs[i].max(0)
  #     new_labels.append(new_outputs_numpy[i,move])
#       print move, move == NUM_ACTIONS - 1
      move = move.data.cpu().numpy()
      invalid = env_states[i].move(int(move/BOARD_SIZE), move%BOARD_SIZE, passMove = (move == (NUM_ACTIONS-1)))
      if(invalid==1):
        legal_moves = env_states[i].find_legal_moves()
        if(len(legal_moves)==0):
          move = NUM_ACTIONS-1
        else:
          move = legal_moves[np.random.randint(len(legal_moves))]
        env_states[i].move(int(move/BOARD_SIZE), move%BOARD_SIZE, passMove = (move == NUM_ACTIONS-1))
        
    hasEnded.append(env_states[i].has_ended())
#   new_labels_numpy = np.array(new_labels)
#   new_labels = Variable(torch.tensor(new_labels))
    
#   with torch.no_grad():
  saved_states = convertEnvToOpponentStates(env_states)
  new_inputs = Variable(torch.tensor(saved_states, dtype=torch.float).cuda())
  new_outputs = model(new_inputs)
    
#   new_outputs_numpy = new_outputs.data.cpu().numpy()
  
  targets = []
  for i in range(BATCH_SIZE):
    if(hasEnded[i] or hadEnded[i]):
      targets.append(rewards[i])  
    else:
      val = new_outputs[i].max()
#       print rewards[i]
      targets.append(val.data.cpu().numpy()*discount_factor + rewards[i].data.cpu().numpy())
    
  keepIndices = []
  for i in range(BATCH_SIZE):
    if(hasEnded[i] or hadEnded[i]):
      continue
    else:
      keepIndices.append(i)
      
  saved_states = saved_states[keepIndices]
#   saved_env_states = env_states[keepIndices]
  saved_env_states = [env_states[i] for i in keepIndices]
    
  targetsNumpy = np.array(targets)
  targets = Variable(torch.tensor(targetsNumpy, dtype=torch.float).cuda())
  
#   print "HERE"
  loss = criterion(labels, targets)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
#   if (i+1) % 100 == 0:
#   print ('Epoch [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, NUM_EPOCHS, loss.item()))
#   print_usage()
  
  if((epoch+1) % 50 == 0):
    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, NUM_EPOCHS, loss.item()))
    torch.save(model.state_dict(), './saved_dqn_cnn-updated/dsavedqn_cnn_epoch_{}.pth'.format(epoch))
    
# torch.save(model.state_dict(), './saved_dqn_cnn/dqn_cnn_epoch_{}.pth'.format(epoch))

In [ ]:
print NUM_ACTIONS